# ADA development

- [x] data type detection.
- [x] format column name
- [x] extract datetime elements.
- [ ] encapsular en una clase data para prepara el df input
    - [ ] devolver datos por nombre de columna.

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
# datasets
import seaborn

## dataset

In [91]:
df = seaborn.load_dataset('titanic')
df.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

In [92]:
df.dtypes

survived          int64
pclass            int64
sex              object
age             float64
sibsp             int64
parch             int64
fare            float64
embarked         object
class          category
who              object
adult_male         bool
deck           category
embark_town      object
alive            object
alone              bool
dtype: object

## type of columns

In [82]:
from dateutil.parser import parse

class Data():
    def __init__(self, df):
        self.df = df
        self.columns = dict()
        # run
        self.__run()
        # clean
        del df
    
    # get column(s) data
    def get(self, column_name: 'str or list of str', clean = False):
        if clean:
            return self.df[column_name].dropna().values
        else:
            return self.df[column_name].values
        
    # run
    def __run(self):
        # check datetime columns and format if there are
        self.__datetime_format()
        # other types
        num_cols = [col for col in self.df.select_dtypes(include=['float64']).columns.values if not col in datetime_cols]
        disc_cols = [col for col in self.df.select_dtypes(include=['int64', 'bool']).columns.values if not col in datetime_cols]
        cat_cols = [col for col in self.df.select_dtypes(include=['object', 'category']).columns.values if not col in datetime_cols]
        # store types
        self.columns['numerical'] = num_cols
        self.columns['discrete'] = disc_cols
        self.columns['categorical'] = cat_cols
        # format column names
        self.__format_column_name()
        
    # datetime detection
    def __datetime_format(self):
        # parse datetime values
        def parse_dt(dt):
            try:
                return parse(dt)
            except:
                return np.nan
        # initialize
        datetime_cols = list()
        # loop of columns
        for col in self.df.columns.tolist():
            data = self.df[col].values
            dt = np.array(list(map(lambda dt: parse_dt(dt), data)))
            # check if column is datetime
            if len(dt)==len(list(filter(lambda v: v==v, dt))):
                print('"%s" is datetime'%col)
                # conversion if it is datetime
                self.df[col] = dt
                # store column name
                datetime_cols.append(col)
            else:
                pass
        # extract datetime elements
        for col in datetime_cols:
            self.data.df['%s_year'] = self.data.df.dt.year
            self.data.df['%s_month'] = self.data.df.dt.month
            self.data.df['%s_day'] = self.data.df.dt.day
            self.data.df['%s_hour'] = self.data.df.dt.hour
            self.data.df['%s_doy'] = self.data.df.dt.dayofyear
            self.data.df['%s_dow'] = self.data.df.dt.dayofweek
        # set type of columns 
        self.columns['datetime'] = datetime_cols
        
    # format column name
    def __format_column_name(self):
        dc = {'numerical':'N', 'discrete':'D', 'categorical':'C', 'datetime':'DT'}
        for col_type, col_names in self.columns.items():
            columns = list()
            for col in col_names:
                col_new = '%s_%s'%(dc[col_type], str.lower(col.replace(' ', '_')))
                columns.append(col_new)
                self.df.rename(columns = {col:col_new}, inplace = True)
            self.columns[col_type] = columns

In [93]:
data = Data(df.copy())

In [94]:
data.columns      

{'datetime': [],
 'numerical': ['N_age', 'N_fare'],
 'discrete': ['D_survived',
  'D_pclass',
  'D_sibsp',
  'D_parch',
  'D_adult_male',
  'D_alone'],
 'categorical': ['C_sex',
  'C_embarked',
  'C_class',
  'C_who',
  'C_deck',
  'C_embark_town',
  'C_alive']}

In [95]:
data.df.head()

,D_survived,D_pclass,C_sex,N_age,D_sibsp,D_parch,N_fare,C_embarked,C_class,C_who,D_adult_male,C_deck,C_embark_town,C_alive,D_alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [96]:
data.get(['C_who', 'C_deck'], clean = True)

array([['woman', 'C'],
       ['woman', 'C'],
       ['man', 'E'],
       ['child', 'G'],
       ['woman', 'C'],
       ['man', 'D'],
       ['man', 'A'],
       ['man', 'C'],
       ['woman', 'B'],
       ['woman', 'D'],
       ['man', 'B'],
       ['man', 'C'],
       ['woman', 'B'],
       ['man', 'C'],
       ['woman', 'F'],
       ['man', 'F'],
       ['woman', 'C'],
       ['man', 'E'],
       ['man', 'A'],
       ['man', 'D'],
       ['man', 'D'],
       ['man', 'C'],
       ['man', 'B'],
       ['woman', 'E'],
       ['man', 'D'],
       ['woman', 'F'],
       ['woman', 'D'],
       ['man', 'C'],
       ['man', 'B'],
       ['man', 'F'],
       ['woman', 'C'],
       ['woman', 'E'],
       ['man', 'B'],
       ['man', 'A'],
       ['woman', 'C'],
       ['child', 'F'],
       ['man', 'A'],
       ['child', 'F'],
       ['woman', 'B'],
       ['woman', 'B'],
       ['child', 'G'],
       ['man', 'A'],
       ['woman', 'D'],
       ['woman', 'D'],
       ['man', 'C'],
       ['wo